In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, distance
import cv2
import numpy as np
import matplotlib.cm as cm
from sklearn.metrics import pairwise_distances_argmin_min
import matplotlib.lines as mlines
import string
import matplotlib.patches as patches

In [ ]:
column_names = ['frame','vehicle_id','vehicle_type','a','b','c','X1','Y1','X2','Y2',]
# df = pd.read_csv("C:\\Users\\ancha\\Box\\QC-5900\\Data\\project2\\16393238\\segments\\16393238-cam_1-2-north_west-20231109_0000-010_1699916237331_out\\16393238-cam_1-2-north_west-20231109_0000-010_1699916237331.txt", sep=' ', header=None, usecols = [0,1,2,3,4,5,6,7,8,9], names=column_names)
df = pd.read_csv("C:\\Users\\ancha\\Box\\QC-5900\\Data\\project2\\16393239\\segments\\16393239-cam_1-3-south_east-20231109_0000-010_1699928799703_out\\16393239-cam_1-3-south_east-20231109_0000-010_1699928799703.txt", sep=' ', header=None, usecols = [0,1,2,3,4,5,6,7,8,9], names=column_names)
# df = pd.read_csv("C:\\Users\\ancha\\Box\\QC-5900\\Data\\project2\\16393240\\segments\\16393240-cam_1-2-south_west-20231109_0000-010_1699922657350_out\\16393240-cam_1-2-south_west-20231109_0000-010_1699922657350.txt", sep=' ', header=None, usecols = [0,1,2,3,4,5,6,7,8,9], names=column_names)
# df = pd.read_csv("C:\\Users\\ancha\\Box\\QC-5900\\Data\\project2\\16393241\\segments\\16393241-cam_1-2-south_west-20231109_0000-010_1699916981923_out\\16393241-cam_1-2-south_west-20231109_0000-010_1699916981923.txt", sep=' ', header=None, usecols = [0,1,2,3,4,5,6,7,8,9], names=column_names)
# df = pd.read_csv("C:\\Users\\ancha\\Box\\QC-5900\\Data\\project2\\16393242\\segments\\16393242-cam_1-4-south_west-20231109_0000-010_1700066770343_out\\16393242-cam_1-4-south_west-20231109_0000-010_1700066770343.txt", sep=' ', header=None, usecols = [0,1,2,3,4,5,6,7,8,9], names=column_names)

df['Center_X'] = (df['X1']+df['X2']) / 2
df['Center_Y'] = (df['Y1']+df['Y2']) / 2

df = df[['frame','vehicle_id','vehicle_type','Center_X','Center_Y']]

In [ ]:
print(df.shape)
df_unique = df.nunique()
df_unique

In [ ]:
# image_path = "C:\\Users\\ancha\\Box\\QC-5900\\Data\\project2\\16393238\\16393238-cam_1-2-north_west-20231109_0000_1699916237331_out\\16393238-cam_1-2-north_west-20231109_0000_1699916237331_frame.jpg"
image_path = "C:\\Users\\ancha\\Box\\QC-5900\\Data\\project2\\16393239\\16393239-cam_1-3-south_east-20231109_0000_1699928799703_out\\16393239-cam_1-3-south_east-20231109_0000_1699928799703_frame.jpg"
# image_path = "C:\\Users\\ancha\\Box\\QC-5900\\Data\\project2\\16393240\\segments\\16393240-cam_1-2-south_west-20231109_0000-010_1699922657350_out\\16393240-cam_1-2-south_west-20231109_0000-010_1699922657350_frame.jpg"
# image_path = "C:\\Users\\ancha\\Box\\QC-5900\\Data\\project2\\16393241\\segments\\16393241-cam_1-2-south_west-20231109_0000-010_1699916981923_out\\16393241-cam_1-2-south_west-20231109_0000-010_1699916981923_frame.jpg"
# image_path = "C:\\Users\\ancha\\Box\\QC-5900\\Data\\project2\\16393242\\segments\\16393242-cam_1-4-south_west-20231109_0000-010_1700066770343_out\\16393242-cam_1-4-south_west-20231109_0000-010_1700066770343_frame.jpg"


In [ ]:
df_sorted = df.sort_values(by=['vehicle_id','frame']).reset_index(drop=True)
# df_sorted = df_sorted.groupby('vehicle_id').filter(lambda x: (len(x) >= 15))

# df_sorted = df_sorted.groupby('vehicle_id').filter(
#     lambda x: (abs(x['Center_X'].iloc[0] - x['Center_X'].iloc[-1]) > 100) |
#               (abs(x['Center_Y'].iloc[0] - x['Center_Y'].iloc[-1]) > 100)
# )

df_sorted = df_sorted.groupby('vehicle_id').filter(
    lambda x: np.sqrt((x['Center_X'].iloc[-1] - x['Center_X'].iloc[0])**2 +
                      (x['Center_Y'].iloc[-1] - x['Center_Y'].iloc[0])**2) > 100
)

# df_sorted.to_csv("../testfile.csv")
# for vehicle_id, group in df_sorted.groupby('vehicle_id'):
#     count = ((abs(group['dx']) < 0.5) & (abs(group['dy']) < 0.5)).sum()
#     print(f"Vehicle ID: {vehicle_id}, Count: {count}")
    

print(df_sorted)
# # Filter based on mean dx and dy values
# df_sorted = df_sorted.groupby('vehicle_id').filter(
#     lambda x: abs(x['dx'].mean()) > 0.5 and abs(x['dy'].mean()) > 0.5
# )
df_first_frame = df_sorted.groupby('vehicle_id').head(1).reset_index()
df_last_frame = df_sorted.groupby('vehicle_id').tail(1).reset_index()
df_combined = pd.concat([df_first_frame, df_last_frame]).sort_values(by='vehicle_id').reset_index()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.imshow(cv2.imread(image_path))
# Group data by 'vehicle_id' and plot trajectory lines
for vehicle_id, group in df_sorted.groupby('vehicle_id'):
    max_x_travelled = abs(group['Center_X'].iloc[0] - group['Center_X'].iloc[-1])
    max_y_travelled = abs(group['Center_X'].iloc[0] - group['Center_X'].iloc[-1])
    # print(f"Vehicle ID: {vehicle_id}, max_x_travelled: {max_x_travelled}, max_y_travelled: {max_y_travelled}")
    # cluster_color = plt.get_cmap("viridis")(group['InCluster'].iloc[0]/df['InCluster'].max())
    ax.plot(group['Center_X'], group['Center_Y'], c='red')

In [ ]:

# test = df_sorted.groupby('vehicle_id')[['dx', 'dy']].mean()
# moving_vehicle_ids = test[(test['dx'].abs()>0.2) & (test['dy'].abs()>0.2)].index
# # Filter df_sorted to keep only rows where vehicle_id is in moving_vehicle_ids
# df_sorted = df_sorted[df_sorted['vehicle_id'].isin(moving_vehicle_ids)]

# print(test[(test['dx'].abs()>0.2) & (test['dy'].abs()>0.2)])

# print(df_sorted[df_sorted['dx'] == 0])
print(df_sorted.shape)

In [ ]:
df_sorted['dx'] = df_sorted.groupby('vehicle_id')['Center_X'].diff(500)
df_sorted['dy'] = df_sorted.groupby('vehicle_id')['Center_Y'].diff(500)

# # Filter based on count of dx and dy values
# parking = df_sorted.groupby('vehicle_id').filter(
#     lambda x: (((abs(x['dx']) < 50) & (abs(x['dy']) < 50)).sum() > 100)
# )
parking = df_sorted[(abs(df_sorted['dx'])<5)|(abs(df_sorted['dy'])<5)]

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 8))
# for vehicle_id, group in df_sorted.groupby('vehicle_id'):
#     ax.plot(group['Center_X'], group['Center_Y'], c = 'red')
#     ax.imshow(cv2.imread(image_path))

# fig, ax = plt.subplots(figsize=(10, 8))
# abc = df_sorted.merge(df_first_frame['vehicle_id'], on='vehicle_id', how='inner')
# for vehicle_id, group in df_first_frame.groupby('vehicle_id'):
#     ax.scatter(group['Center_X'], group['Center_Y'], c = 'red')
#     ax.imshow(cv2.imread(image_path))

# fig, ax = plt.subplots(figsize=(10, 8))
# abc = df_sorted.merge(df_last_frame['vehicle_id'], on='vehicle_id', how='inner')
# for vehicle_id, group in df_last_frame.groupby('vehicle_id'):
#     ax.scatter(group['Center_X'], group['Center_Y'], c = 'red')
#     ax.imshow(cv2.imread(image_path))


In [ ]:
print(df_first_frame.shape)
print(df_last_frame.shape)

In [ ]:

# test = df_sorted.groupby('vehicle_id')[['dx', 'dy']].mean()
# moving_vehicle_ids = test[(test['dx'].abs()>0.2) & (test['dy'].abs()>0.2)].index
# # Filter df_sorted to keep only rows where vehicle_id is in moving_vehicle_ids
# df_sorted = df_sorted[df_sorted['vehicle_id'].isin(moving_vehicle_ids)]

# print(test[(test['dx'].abs()>0.2) & (test['dy'].abs()>0.2)])

# print(df_sorted[df_sorted['dx'] == 0])
print(df_sorted.shape)

In [ ]:
if(df_first_frame.shape[0] < 25):
    min_samples_val = 2
elif(df_first_frame.shape[0] < 100):
    min_samples_val = 5
else: min_samples_val = 20

In [ ]:
# Assume df_first contains the first occurrence of each vehicle with 'x_center' and 'y_center'
# normalize X, Y coordinates
df_first_frame['Center_X_norm'] = (df_first_frame['Center_X'] - df_first_frame['Center_X'].mean()) / df_first_frame['Center_X'].std()
df_first_frame['Center_Y_norm'] = (df_first_frame['Center_Y'] - df_first_frame['Center_Y'].mean()) / df_first_frame['Center_Y'].std()

# Step 1: Extract the coordinates
X_first = df_first_frame[['Center_X_norm', 'Center_Y_norm']].values

# Step 2: Apply DBSCAN
# adjust epsilon ('eps') and min_samples based on data scale.
db = DBSCAN(eps=0.2, min_samples=min_samples_val)  # eps is the maximum distance between two samples for them to be considered neighbors

# Step 3: Fit the DBSCAN model
df_first_frame['Cluster'] = db.fit_predict(X_first)
df_outlier = df_first_frame[df_first_frame['Cluster'] == -1].reset_index(drop=True)
df_first = df_first_frame[df_first_frame['Cluster'] != -1].reset_index(drop=True)

# Compute centroid of each cluster
centroids = df_first.groupby('Cluster')[['Center_X', 'Center_Y']].mean().reset_index()

# Display centroids
# print(centroids)

# Display no. of vehicles in each cluster
print(df_first['Cluster'].value_counts())
print(df_outlier.shape)



In [ ]:
unique_clusters = df_first['Cluster'].unique()
# Create a dynamic mapping from cluster ID to gate name
cluster_to_gate = {cluster_id: f'Gate {i+1}' for i, cluster_id in enumerate(unique_clusters)}
print(cluster_to_gate)
# Assign the corresponding gate names to the 'Gate' column
df_first['InGate'] = df_first['Cluster'].map(cluster_to_gate)

In [ ]:
df_first_clusters = df_first.copy()

# Store updated clusters
updated_clusters = []
fig, ax = plt.subplots(figsize=(10, 8))
for cluster_id, group in df_first_clusters.groupby('Cluster'):
    # centroid = group[['Center_X','Center_Y']].mean().values
    centroid = (round(float(group['Center_X'].mean()),2), round(float(group['Center_Y'].mean())))
    cluster_count = len(group)
    print(cluster_id, ":", centroid, ":", cluster_count)


    # Find the extreme points
    x_min = group.loc[group['Center_X'].idxmin(), 'Center_X']  # Leftmost point
    x_max = group.loc[group['Center_X'].idxmax(), 'Center_X']  # Rightmost point
    y_min = group.loc[group['Center_Y'].idxmin(), 'Center_Y']  # Bottommost point
    y_max = group.loc[group['Center_Y'].idxmax(), 'Center_Y']  # Topmost point
    
    # # Identify outliers that are outside the cluster boundary
    # outliers_outside_boundary = df_outlier[
    #     (df_outlier['Center_X'] < x_min) | (df_outlier['Center_X'] > x_max) |
    #     (df_outlier['Center_Y'] < y_min) | (df_outlier['Center_Y'] > y_max)
    # ]
        
    # Define rectangle boundary
    rectangle_corners = np.array([
        [x_min, y_max],  # Top-left
        [x_max, y_max],  # Top-right
        [x_max, y_min],  # Bottom-right
        [x_min, y_min],  # Bottom-left
        [x_min, y_max]   # Close the rectangle
    ])

    # Plot the rectangle
    plt.plot(rectangle_corners[:, 0], rectangle_corners[:, 1], 'k-', linewidth=2)

    plt.scatter(df_outlier['Center_X'], df_outlier['Center_Y'], c = df_outlier['Cluster'])
    
    # Get InGate value using the cluster_id
    in_gate = cluster_to_gate.get(cluster_id, 'Unknown')  # Default to 'Unknown' if cluster_id not in the mapping

    # Append updated cluster data
    updated_clusters.append({
        'InClusterId': cluster_id,
        'InGate': in_gate,
        'Centroid': (centroid[0], centroid[1]),
        'Boundary': (x_min, x_max, y_min, y_max)
    })
    
    # Plot cluster points
    plt.scatter(group['Center_X'], group['Center_Y'], label=f'Cluster {cluster_id}')

    # Plot gate name near the cluster boundary (placing near top-right corner)
    plt.text(x_min + 5, y_min - 5, f'{in_gate}', fontsize=12, color='red', fontweight='bold')


    # Plot centroid
    plt.scatter(centroid[0], centroid[1], color='red', marker='x', s=100, label=f'Centroid {cluster_id}')

    ax.imshow(cv2.imread(image_path))
print(updated_clusters)
InGateClusters = pd.DataFrame(updated_clusters).sort_values('InClusterId').reset_index(drop=True)
print(InGateClusters)


In [ ]:
# Assume df_last contains the last occurrence of each vehicle with 'x_center' and 'y_center'
# normalize X, Y coordinates
df_last_frame['Center_X_norm'] = (df_last_frame['Center_X'] - df_last_frame['Center_X'].mean()) / df_last_frame['Center_X'].std()
df_last_frame['Center_Y_norm'] = (df_last_frame['Center_Y'] - df_last_frame['Center_Y'].mean()) / df_last_frame['Center_Y'].std()

# Step 1: Extract the coordinates
X_last = df_last_frame[['Center_X_norm', 'Center_Y_norm']].values

# Step 2: Apply DBSCAN
# adjust epsilon ('eps') and min_samples based on data scale.
db = DBSCAN(eps=0.2, min_samples=min_samples_val)  # eps is the maximum distance between two samples for them to be considered neighbors

# Step 3: Fit the DBSCAN model
df_last_frame['Cluster'] = db.fit_predict(X_last)
df_outlier_last = df_last_frame[df_last_frame['Cluster'] == -1].reset_index(drop=True)
df_last = df_last_frame[df_last_frame['Cluster'] != -1].reset_index(drop=True)


# Compute centroid of each cluster
centroids = df_last.groupby('Cluster')[['Center_X', 'Center_Y']].mean().reset_index()

# # Display centroids
# print(centroids)

# Display no. of vehicles in each cluster
print(df_last['Cluster'].value_counts())
print(df_outlier_last.shape)





In [ ]:
unique_clusters = df_last['Cluster'].unique()

# Generate labels A, B, C, ... for clusters
cluster_labels = list(string.ascii_uppercase)  # ['A', 'B', 'C', ..., 'Z']

# Ensure we don't exceed available labels
if len(unique_clusters) > len(cluster_labels):
    raise ValueError("Too many clusters, not enough labels!")

# Assign gate names using letters instead of numbers
cluster_to_gate = {cluster_id: f'Gate {cluster_labels[i]}' for i, cluster_id in enumerate(unique_clusters)}

print(cluster_to_gate)

# Assign the corresponding gate names to the 'Gate' column
df_last['OutGate'] = df_last['Cluster'].map(cluster_to_gate)

In [ ]:
# Copy df_first to avoid modifying the original during iteration
# # Define the OutGate mapping dictionary
# out_gate_mapping = {0: 'W', 1: 'E', 2: 'S', 3: 'E', 4: 'S'}

df_last_clusters = df_last.copy()

# Store updated clusters
updated_clusters = []
fig, ax = plt.subplots(figsize=(10, 8))
for cluster_id, group in df_last_clusters.groupby('Cluster'):
    centroid = (round(float(group['Center_X'].mean()),2), round(float(group['Center_Y'].mean())))
    cluster_count = len(group)
    print(cluster_id, ":", centroid, ":", cluster_count)
    
    # Find the extreme points
    x_min = group.loc[group['Center_X'].idxmin(), 'Center_X']  # Leftmost point
    x_max = group.loc[group['Center_X'].idxmax(), 'Center_X']  # Rightmost point
    y_min = group.loc[group['Center_Y'].idxmin(), 'Center_Y']  # Bottommost point
    y_max = group.loc[group['Center_Y'].idxmax(), 'Center_Y']  # Topmost point


    # Identify outliers that are outside the cluster boundary
    outliers_outside_boundary = df_outlier_last[
        (df_outlier_last['Center_X'] < x_min) | (df_outlier_last['Center_X'] > x_max) |
        (df_outlier_last['Center_Y'] < y_min) | (df_outlier_last['Center_Y'] > y_max)
    ]
    

    # Define rectangle boundary
    rectangle_corners = np.array([
        [x_min, y_max],  # Top-left
        [x_max, y_max],  # Top-right
        [x_max, y_min],  # Bottom-right
        [x_min, y_min],  # Bottom-left
        [x_min, y_max]   # Close the rectangle
    ])

    # Plot the rectangle
    plt.plot(rectangle_corners[:, 0], rectangle_corners[:, 1], 'k-', linewidth=2)

    plt.scatter(df_outlier_last['Center_X'], df_outlier_last['Center_Y'], c = df_outlier_last['Cluster'])

    # Get InGate value using the cluster_id
    out_gate = cluster_to_gate.get(cluster_id, 'Unknown')  # Default to 'Unknown' if cluster_id not in the mapping
    print(cluster_id, out_gate)
    # Append updated cluster data
    updated_clusters.append({
        'OutClusterId': cluster_id,
        'OutGate': out_gate,  # Added InGate value
        'Centroid': centroid,
        'Boundary': (x_min, x_max, y_min, y_max)
    })

    # Plot cluster points
    plt.scatter(group['Center_X'], group['Center_Y'], label=f'Cluster {cluster_id}')

    # Plot gate name near the cluster boundary (placing near top-right corner)
    plt.text(x_min + 5, y_min - 5, f'{out_gate}', fontsize=12, color='red', fontweight='bold')

    # Plot centroid
    plt.scatter(centroid[0], centroid[1], color='red', marker='x', s=100, label=f'Centroid {cluster_id}')

    ax.imshow(cv2.imread(image_path))

OutGateClusters = pd.DataFrame(updated_clusters).sort_values('OutClusterId').reset_index(drop=True)
print(OutGateClusters)

# test = df_sorted.merge(df_last_clusters[df_last_clusters['Cluster'] == 0][['vehicle_id','Cluster']],on='vehicle_id',how='inner')
# print(test['Cluster'].max())
# fig, ax = plt.subplots(figsize=(10, 8))
# ax.imshow(cv2.imread(image_path))
# # Group data by 'vehicle_id' and plot trajectory lines
# for vehicle_id, group in test.groupby('vehicle_id'):
#     # cluster_color = plt.get_cmap("viridis")(group['Cluster'].iloc[0]/test['Cluster'].max())
#     ax.plot(group['Center_X'], group['Center_Y'], c='red')



In [ ]:
df_first_clusters = df_first_clusters.rename(columns={'Cluster':'InCluster'})
df_last_clusters = df_last_clusters.rename(columns={'Cluster':'OutCluster'})
print(df_first_clusters.head(2))
df_first_last = df_first_clusters[['vehicle_id','InCluster','InGate']].merge(df_last_clusters[['vehicle_id','OutCluster','OutGate']],on='vehicle_id',how='outer')
print(df_first_last.nunique())

In [ ]:
print(df_first_last[df_first_last['InCluster'].isnull()].shape)
print(df_first_last[df_first_last['OutCluster'].isnull()].shape)
print(df_first_last[(df_first_last['OutCluster'].isnull())&(df_first_last['InCluster'].isnull())].shape)
print(df_first_last[(df_first_last['OutCluster'].notnull())&(df_first_last['InCluster'].notnull())].shape)

In [ ]:
# df_ingate_null = df_first_last[df_first_last['InCluster'].isnull()].merge(df_first_frame[['vehicle_id','Center_X','Center_Y']],on='vehicle_id',how='left')

# # Convert centroids to NumPy array
# centroids = np.array([cluster.Centroid for cluster in InGateClusters.itertuples()])
# df_points = df_ingate_null[['Center_X', 'Center_Y']].values

# # Find closest centroid index and distance
# indices, distances = pairwise_distances_argmin_min(df_points, centroids)

# # Assign the closest cluster and distance
# df_ingate_null['InCluster'] = [InGateClusters.iloc[i].InClusterId for i in indices]
# df_ingate_null['InGate'] = [InGateClusters.iloc[i].InGate for i in indices]
# df_ingate_null['Min_Distance'] = distances

df_first_last = df_first_last.merge(df_first_frame[['vehicle_id', 'Center_X', 'Center_Y']], 
                                        on='vehicle_id', how='left')
ingate_updated_rows = pd.DataFrame()
# Identify rows where InCluster is null
mask = df_first_last['InCluster'].isnull()
# Convert centroids to NumPy array
centroids = np.array([cluster.Centroid for cluster in InGateClusters.itertuples()])
df_points = df_first_last.loc[mask, ['Center_X', 'Center_Y']].values

if len(df_points) > 0:
    # Find closest centroid index and distance
    indices, distances = pairwise_distances_argmin_min(df_points, centroids)
    
    # Update df_first_last in place
    df_first_last.loc[mask, 'InCluster'] = [InGateClusters.iloc[i].InClusterId for i in indices]
    df_first_last.loc[mask, 'InGate'] = [InGateClusters.iloc[i].InGate for i in indices]
    # df_first_last.loc[mask, 'Min_Distance'] = distances

# if len(df_points) > 0:
#     # Find closest centroid index and distance
#     indices, distances = pairwise_distances_argmin_min(df_points, centroids)
#     # Filter out points that are further than 200 from their nearest cluster
#     valid_indices = distances <= 100
#     # Get the indices where valid_indices is True
#     valid_indices_positions = np.where(valid_indices)[0]
#     print(valid_indices_positions)
#     if len(valid_indices_positions) > 0:  # Proceed only if there are valid points
#         # Make sure to apply the mask & valid_indices correctly
#         valid_mask = np.zeros(len(df_first_last), dtype=bool)
#         # Mark positions corresponding to valid indices as True in the valid_mask
#         valid_mask[valid_indices_positions] = True
#         print(valid_mask)
#         # Update df_first_last in place
#         df_first_last.loc[valid_mask, 'InCluster'] = [InGateClusters.iloc[i].InClusterId for i in indices[valid_indices_positions]]
#         df_first_last.loc[valid_mask, 'InGate'] = [InGateClusters.iloc[i].InGate for i in indices[valid_indices_positions]]
#         # df_first_last.loc[mask, 'Min_Distance'] = distances

else:
    print("No rows to update.")
    
# Drop 'Center_X' and 'Center_Y' after updating
df_first_last.drop(columns=['Center_X', 'Center_Y'], inplace=True, errors='ignore')
# print(df_first_last[mask])

# Print only the rows that were updated
ingate_updated_rows = df_first_last.loc[mask]
print(ingate_updated_rows.shape)




In [ ]:
# df_ingate_null = df_ingate_null.merge(
#     OutGateClusters[['OutClusterId', 'OutGate']], 
#     how='left', 
#     left_on='OutCluster', 
#     right_on='OutClusterId'
# )

# # Merge with InGateClusters to get InGate
# df_ingate_null = df_ingate_null.merge(
#     InGateClusters[['InClusterId', 'InGate']], 
#     how='left', 
#     left_on='InCluster', 
#     right_on='InClusterId'
# )

if not ingate_updated_rows.empty:
    # Display the updated df_ingate_null dataframe
    print((ingate_updated_rows[['InCluster','InGate','OutCluster','OutGate']].value_counts().reset_index()))
# print((df_ingate_null[(df_ingate_null['InCluster'] == 2) & (df_ingate_null['OutCluster'] == 2)][['InCluster','OutCluster','Center_X','Center_Y']]))

In [ ]:
# Initialize a dictionary to store legend handles
legend_handles = {}

if not ingate_updated_rows.empty:
    # Merge df_ingate_null with InGateClusters to get OutGate
    df = df_sorted.merge(ingate_updated_rows[['vehicle_id','InCluster','OutCluster','InGate','OutGate']],on='vehicle_id',how='inner')
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.imshow(cv2.imread(image_path))
    # Get unique ('InGate', 'OutGate') pairs
    unique_clusters = df[['InGate', 'OutGate']].drop_duplicates()
    unique_clusters = [tuple(x) for x in unique_clusters.to_numpy()]  # Convert to list of tuples

    # Create a colormap
    cmap = cm.get_cmap('tab10', len(unique_clusters))

    # Create a mapping from ('InGate', 'OutGate') to a color
    color_mapping = {cluster_pair: cmap(i / len(unique_clusters)) for i, cluster_pair in enumerate(unique_clusters)}



# # Plot InGate locations with labels
# for _, row in InGateClusters.iterrows():
#     centroid_x, centroid_y = row['Centroid']
#     ax.scatter(centroid_x, centroid_y, color='blue', marker='o', s=100, label=row['InGate'])
#     ax.text(centroid_x, centroid_y, row['InGate'], fontsize=10, ha='center', va='center', color='white', bbox=dict(facecolor='blue', alpha=0.6, edgecolor='black'))

# # Plot OutGate locations with labels
# for _, row in OutGateClusters.iterrows():
#     centroid_x, centroid_y = row['Centroid']
#     ax.scatter(centroid_x, centroid_y, color='red', marker='o', s=100, label=row['OutGate'])
#     ax.text(centroid_x, centroid_y, row['OutGate'], fontsize=10, ha='center', va='center', color='white', bbox=dict(facecolor='red', alpha=0.6, edgecolor='black'))

    # Plot InGate bounding boxes with labels
    for _, row in InGateClusters.iterrows():
        x_min, x_max, y_min, y_max = row['Boundary']
        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='blue', facecolor='none')
        ax.add_patch(rect)
        ax.text((x_min + x_max) / 2, (y_min + y_max) / 2, row['InGate'], fontsize=10, ha='center', va='center', 
                color='white', bbox=dict(facecolor='blue', alpha=0.6, edgecolor='black'))

    # Plot OutGate bounding boxes with labels
    for _, row in OutGateClusters.iterrows():
        x_min, x_max, y_min, y_max = row['Boundary']
        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)
        ax.text((x_min + x_max) / 2, (y_min + y_max) / 2, row['OutGate'], fontsize=10, ha='center', va='center', 
                color='white', bbox=dict(facecolor='red', alpha=0.6, edgecolor='black'))
        

    # Group data by 'vehicle_id' and plot trajectory lines
    for vehicle_id, group in df.groupby('vehicle_id'):
        # Extract the unique ('InGate', 'OutGate') pair for this vehicle
        cluster_pair = (group['InGate'].iloc[0], group['OutGate'].iloc[0])

        # Get the corresponding color
        color = color_mapping.get(cluster_pair, 'black')  # Default to black if not found

        ax.plot(group['Center_X'], group['Center_Y'], color = color, label=f'{cluster_pair[0]} → {cluster_pair[1]}')
        # Store legend entry only once per unique cluster pair
        if cluster_pair not in legend_handles:
            legend_handles[cluster_pair] = mlines.Line2D([], [], color=color, label=f'{cluster_pair[0]} → {cluster_pair[1]}')

    # Add legend with correct color mapping
    ax.legend(handles=legend_handles.values(), loc='best', fontsize='small', markerscale=0.5)

    # Optional: Add legend to show cluster pairs
    # ax.legend([f'Cluster {pair[0]} → {pair[1]}' for pair in unique_clusters], loc='best', fontsize='small', markerscale=0.5)


In [ ]:
df_first_last = df_first_last.merge(df_last_frame[['vehicle_id', 'Center_X', 'Center_Y']], 
                                        on='vehicle_id', how='left')
outgate_updated_rows = pd.DataFrame()

# Identify rows where OutCluster is null
mask = df_first_last['OutCluster'].isnull()
# Convert centroids to NumPy array
centroids = np.array([cluster.Centroid for cluster in OutGateClusters.itertuples()])
df_points = df_first_last.loc[mask, ['Center_X', 'Center_Y']].values

if len(df_points) > 0:
    # Find closest centroid index and distance
    indices, distances = pairwise_distances_argmin_min(df_points, centroids)
    
    # Update df_first_last in place
    df_first_last.loc[mask, 'OutCluster'] = [OutGateClusters.iloc[i].OutClusterId for i in indices]
    df_first_last.loc[mask, 'OutGate'] = [OutGateClusters.iloc[i].OutGate for i in indices]
    # df_first_last.loc[mask, 'Min_Distance'] = distances
    

else:
    print("No rows to update.")

# Drop 'Center_X' and 'Center_Y' after updating
df_first_last.drop(columns=['Center_X', 'Center_Y'], inplace=True, errors='ignore')
# print(df_first_last[mask])

# Print only the rows that were updated
outgate_updated_rows = df_first_last.loc[mask]
print(outgate_updated_rows.shape)

In [ ]:
# df_outgate_null = df_outgate_null.merge(
#     OutGateClusters[['OutClusterId', 'OutGate']], 
#     how='left', 
#     left_on='OutCluster', 
#     right_on='OutClusterId'
# )

# # Merge with InGateClusters to get InGate
# df_outgate_null = df_outgate_null.merge(
#     InGateClusters[['InClusterId', 'InGate']], 
#     how='left', 
#     left_on='InCluster', 
#     right_on='InClusterId'
# )
if not outgate_updated_rows.empty:
    # Display the updated df_ingate_null dataframe
    print((outgate_updated_rows[['InCluster','InGate','OutCluster','OutGate']].value_counts().reset_index()))
# print((df_outgate_null[(df_outgate_null['InCluster'] == 2) & (df_outgate_null['OutCluster'] == 2)][['InCluster','OutCluster','Center_X','Center_Y']]))

In [ ]:
import matplotlib.lines as mlines

# Initialize a dictionary to store legend handles
legend_handles = {}
if not ingate_updated_rows.empty:

    # Merge df_outgate_null with OutGateClusters to get OutGate
    df = df_sorted.merge(outgate_updated_rows[['vehicle_id','InCluster','OutCluster','InGate','OutGate']],on='vehicle_id',how='inner')
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.imshow(cv2.imread(image_path))
    # Get unique ('InGate', 'OutGate') pairs
    unique_clusters = df[['InGate', 'OutGate']].drop_duplicates()
    unique_clusters = [tuple(x) for x in unique_clusters.to_numpy()]  # Convert to list of tuples

    # Create a colormap
    cmap = cm.get_cmap('tab10', len(unique_clusters))

    # Create a mapping from ('InGate', 'OutGate') to a color
    color_mapping = {cluster_pair: cmap(i / len(unique_clusters)) for i, cluster_pair in enumerate(unique_clusters)}


    # Plot InGate bounding boxes with labels
    for _, row in InGateClusters.iterrows():
        x_min, x_max, y_min, y_max = row['Boundary']
        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='blue', facecolor='none')
        ax.add_patch(rect)
        ax.text((x_min + x_max) / 2, (y_min + y_max) / 2, row['InGate'], fontsize=10, ha='center', va='center', 
                color='white', bbox=dict(facecolor='blue', alpha=0.6, edgecolor='black'))

    # Plot OutGate bounding boxes with labels
    for _, row in OutGateClusters.iterrows():
        x_min, x_max, y_min, y_max = row['Boundary']
        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)
        ax.text((x_min + x_max) / 2, (y_min + y_max) / 2, row['OutGate'], fontsize=10, ha='center', va='center', 
                color='white', bbox=dict(facecolor='red', alpha=0.6, edgecolor='black'))
        
    # Group data by 'vehicle_id' and plot trajectory lines
    for vehicle_id, group in df.groupby('vehicle_id'):
        # Extract the unique ('InGate', 'OutGate') pair for this vehicle
        cluster_pair = (group['InGate'].iloc[0], group['OutGate'].iloc[0])

        # Get the corresponding color
        color = color_mapping.get(cluster_pair, 'black')  # Default to black if not found

        ax.plot(group['Center_X'], group['Center_Y'], color = color, label=f'{cluster_pair[0]} → {cluster_pair[1]}')
        # Store legend entry only once per unique cluster pair
        if cluster_pair not in legend_handles:
            legend_handles[cluster_pair] = mlines.Line2D([], [], color=color, label=f'{cluster_pair[0]} → {cluster_pair[1]}')

    # Add legend with correct color mapping
    ax.legend(handles=legend_handles.values(), loc='best', fontsize='small', markerscale=0.5)

# Optional: Add legend to show cluster pairs
# ax.legend([f'Cluster {pair[0]} → {pair[1]}' for pair in unique_clusters], loc='best', fontsize='small', markerscale=0.5)


In [ ]:
import matplotlib.lines as mlines

# Initialize a dictionary to store legend handles
legend_handles = {}


# Merge df_outgate_null with OutGateClusters to get OutGate
df = df_sorted.merge(df_first_last[(df_first_last['InGate'].notna()) & (df_first_last['OutGate'].notna())][['vehicle_id','InCluster','OutCluster','InGate','OutGate']],on='vehicle_id',how='inner')
fig, ax = plt.subplots(figsize=(10, 8))
ax.imshow(cv2.imread(image_path))
# Get unique ('InGate', 'OutGate') pairs
unique_clusters = df[['InGate', 'OutGate']].drop_duplicates()
unique_clusters = [tuple(x) for x in unique_clusters.to_numpy()]  # Convert to list of tuples

# Create a colormap
cmap = cm.get_cmap('tab10', len(unique_clusters))

# Create a mapping from ('InGate', 'OutGate') to a color
color_mapping = {cluster_pair: cmap(i / len(unique_clusters)) for i, cluster_pair in enumerate(unique_clusters)}


# Plot InGate bounding boxes with labels
for _, row in InGateClusters.iterrows():
    x_min, x_max, y_min, y_max = row['Boundary']
    rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='blue', facecolor='none')
    ax.add_patch(rect)
    ax.text((x_min + x_max) / 2, (y_min + y_max) / 2, row['InGate'], fontsize=10, ha='center', va='center', 
            color='white', bbox=dict(facecolor='blue', alpha=0.6, edgecolor='black'))

# Plot OutGate bounding boxes with labels
for _, row in OutGateClusters.iterrows():
    x_min, x_max, y_min, y_max = row['Boundary']
    rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='red', facecolor='none')
    ax.add_patch(rect)
    ax.text((x_min + x_max) / 2, (y_min + y_max) / 2, row['OutGate'], fontsize=10, ha='center', va='center', 
            color='white', bbox=dict(facecolor='red', alpha=0.6, edgecolor='black'))
    
# Group data by 'vehicle_id' and plot trajectory lines
for vehicle_id, group in df.groupby('vehicle_id'):
    # Extract the unique ('InGate', 'OutGate') pair for this vehicle
    cluster_pair = (group['InGate'].iloc[0], group['OutGate'].iloc[0])

    # Get the corresponding color
    color = color_mapping.get(cluster_pair, 'black')  # Default to black if not found

    ax.plot(group['Center_X'], group['Center_Y'], color = color, label=f'{cluster_pair[0]} → {cluster_pair[1]}')
    # Store legend entry only once per unique cluster pair
    if cluster_pair not in legend_handles:
        legend_handles[cluster_pair] = mlines.Line2D([], [], color=color, label=f'{cluster_pair[0]} → {cluster_pair[1]}')

# Add legend with correct color mapping
ax.legend(handles=legend_handles.values(), loc='best', fontsize='small', markerscale=0.5)

# Optional: Add legend to show cluster pairs
# ax.legend([f'Cluster {pair[0]} → {pair[1]}' for pair in unique_clusters], loc='best', fontsize='small', markerscale=0.5)


In [ ]:
# merges each vehicle's first frame cluster ID into df_sorted, ensuring consistent cluster IDs for all vehicle occurrences.
final_df = df_sorted.merge(df_first_last[['vehicle_id','InCluster','InGate','OutCluster','OutGate']],on='vehicle_id',how='inner')

print(final_df)

In [ ]:
# Store results
vehicle_data_result = []

for vehicle_id, vehicle_traj_group in final_df.groupby('vehicle_id'):
        
    # Store result
    vehicle_data_result.append({
        'Vehicle ID': vehicle_id, 
        'Class': vehicle_traj_group.iloc[0]['vehicle_type'],
        'Start Frame': vehicle_traj_group.iloc[0]['frame'],
        'End Frame': vehicle_traj_group.iloc[-1]['frame'],
        'InCluster': vehicle_traj_group.iloc[0]['InCluster'],
        'InGate': vehicle_traj_group.iloc[0]['InGate'],
        'OutCluster': vehicle_traj_group.iloc[0]['OutCluster'],
        'OutGate': vehicle_traj_group.iloc[0]['OutGate']
        })
        

# Convert to DataFrame if needed
output = pd.DataFrame(vehicle_data_result).sort_values(by='Vehicle ID').reset_index(drop=True)
print(output.shape)

In [ ]:
output['Counted'] = np.where(output['InGate'] == output['OutGate'], '', 'Y')
new_column_order = ['Vehicle ID','Class','Start Frame','End Frame','Counted','InCluster','InGate','OutCluster','OutGate']
output = output[new_column_order]
print(output.nunique())

print(output[['InGate','OutGate']].value_counts().reset_index())
# output.to_csv("../output_40_1.csv",index=False)